In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import clip
from torch.nn import functional as F
import torch.nn as nn
from torchvision import transforms
from PIL import Image


from pathlib import Path 
if Path.cwd().name != "Generation":
    os.chdir("Generation")


train = False
classes = None
pictures= None

data_dir = "../data/things-eeg2"
images_dir = os.path.join(data_dir, "images_set")

def load_data():
    data_list = []
    label_list = []
    texts = []
    images = []
    
    text_directory = os.path.join(images_dir, "training_images" if train else "test_images")

    dirnames = [d for d in os.listdir(text_directory) if os.path.isdir(os.path.join(text_directory, d))]
    dirnames.sort()
    
    if classes is not None:
        dirnames = [dirnames[i] for i in classes]

    for dir in dirnames:

        try:
            idx = dir.index('_')
            description = dir[idx+1:]
        except ValueError:
            print(f"Skipped: {dir} due to no '_' found.")
            continue
            
        new_description = f"{description}"
        texts.append(new_description)

    img_directory = os.path.join(images_dir, "training_images" if train else "test_images")
    
    all_folders = [d for d in os.listdir(img_directory) if os.path.isdir(os.path.join(img_directory, d))]
    all_folders.sort()

    if classes is not None and pictures is not None:
        images = []
        for i in range(len(classes)):
            class_idx = classes[i]
            pic_idx = pictures[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                if pic_idx < len(all_images):
                    images.append(os.path.join(folder_path, all_images[pic_idx]))
    elif classes is not None and pictures is None:
        images = []
        for i in range(len(classes)):
            class_idx = classes[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                images.extend(os.path.join(folder_path, img) for img in all_images)
    elif classes is None:
        images = []
        for folder in all_folders:
            folder_path = os.path.join(img_directory, folder)
            all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            all_images.sort()  
            images.extend(os.path.join(folder_path, img) for img in all_images)
    else:

        print("Error")
    return texts, images
texts, images = load_data()
# images

In [3]:
import os
cp_path = "tmp/train_eeg-epoch=20-VAL__loss=1.8497.ckpt"
cp = torch.load(cp_path)
state_dict = cp["state_dict"]
filtered_state_dict = {k[len("eeg_encoder."):]:v for k, v in state_dict.items() if k.startswith("eeg_encoder.")}
filtered_state_dict

{'logit_scale': tensor(2.6593, device='cuda:0'),
 'encoder.enc_embedding.mask_token': tensor([[ 2.2306, -0.9360,  1.1844, -0.1838, -0.7417, -0.0082, -0.7323, -1.5703,
          -1.2247, -0.2256,  0.7120, -0.2930, -1.3499, -0.7811,  2.0082,  0.2195,
           0.1900,  1.6415,  0.6813,  1.5418, -0.3729,  0.3405,  0.3456,  0.7908,
          -0.8161, -0.6407, -0.9849,  1.4446,  1.4823, -0.3092, -0.1817, -0.2195,
           2.1306, -1.0182, -1.6034, -0.5203,  0.6887,  0.5736, -0.1914, -0.6463,
          -0.1167,  0.1107,  0.0192, -0.7261,  1.7299, -0.1015,  2.2164, -0.0829,
          -2.4172,  1.6279, -0.4375,  0.4521, -1.1272, -0.8642, -1.7898, -0.3008,
           0.7152,  0.8044,  0.9450,  0.8063, -1.2306, -1.9720, -0.4365,  0.7008,
          -0.9247, -0.7822, -0.9294, -3.0381, -0.7363,  1.3152,  0.7837, -0.5545,
          -1.1406, -1.6900, -0.0358,  0.1347, -0.0904, -0.5850,  0.4202,  0.9815,
          -0.8171, -0.9121,  1.0215,  0.6442, -1.9638, -1.0784,  0.8102, -0.0064,
          -0.

In [4]:
import os

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

os.environ["WANDB_API_KEY"] = "KEY"
os.environ["WANDB_MODE"] = 'offline'
from itertools import combinations

import clip
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import tqdm
from eegdatasets_leaveone import EEGDataset

from einops.layers.torch import Rearrange, Reduce

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
import random
from util import wandb_logger
from braindecode.models import EEGNetv4, ATCNet, EEGConformer, EEGITNet, ShallowFBCSPNet
import csv
from torch import Tensor
import itertools
import math
import re
from subject_layers.Transformer_EncDec import Encoder, EncoderLayer
from subject_layers.SelfAttention_Family import FullAttention, AttentionLayer
from subject_layers.Embed import DataEmbedding
import numpy as np
from loss import ClipLoss
import argparse
from torch import nn
from torch.optim import AdamW


class Config:
    def __init__(self):
        self.task_name = 'classification'  # Example task name
        self.seq_len = 250                 # Sequence length
        self.pred_len = 250                # Prediction length
        self.output_attention = False      # Whether to output attention weights
        self.d_model = 250                 # Model dimension
        self.embed = 'timeF'               # Time encoding method
        self.freq = 'h'                    # Time frequency
        self.dropout = 0.25                # Dropout rate
        self.factor = 1                    # Attention scaling factor
        self.n_heads = 4                   # Number of attention heads
        self.e_layers = 1                  # Number of encoder layers
        self.d_ff = 256                    # Dimension of the feedforward network
        self.activation = 'gelu'           # Activation function
        self.enc_in = 63                   # Encoder input dimension (example value)

class iTransformer(nn.Module):
    def __init__(self, configs, joint_train=False,  num_subjects=10):
        super(iTransformer, self).__init__()
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention
        # Embedding
        self.enc_embedding = DataEmbedding(configs.seq_len, configs.d_model, configs.embed, configs.freq, configs.dropout, joint_train=False, num_subjects=num_subjects)
        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        FullAttention(False, configs.factor, attention_dropout=configs.dropout, output_attention=configs.output_attention),
                        configs.d_model, configs.n_heads
                    ),
                    configs.d_model,
                    configs.d_ff,
                    dropout=configs.dropout,
                    activation=configs.activation
                ) for l in range(configs.e_layers)
            ],
            norm_layer=torch.nn.LayerNorm(configs.d_model)
        )

    def forward(self, x_enc, x_mark_enc, subject_ids=None):
        # Embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc, subject_ids)
        enc_out, attns = self.encoder(enc_out, attn_mask=None)
        enc_out = enc_out[:, :63, :]      
        # print("enc_out", enc_out.shape)
        return enc_out

class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=40):
        super().__init__()
        # Revised from ShallowNet
        self.tsconv = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), stride=(1, 1)),
            nn.AvgPool2d((1, 51), (1, 5)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.Conv2d(40, 40, (63, 1), stride=(1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.Dropout(0.5),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  
            Rearrange('b e (h) (w) -> b (h w) e'),
        )

    def forward(self, x: Tensor) -> Tensor:
        # b, _, _, _ = x.shape
        x = x.unsqueeze(1)     
        # print("x", x.shape)   
        x = self.tsconv(x)
        # print("tsconv", x.shape)   
        x = self.projection(x)
        # print("projection", x.shape)  
        return x

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FlattenHead(nn.Sequential):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        return x

class Enc_eeg(nn.Sequential):
    def __init__(self, emb_size=40, **kwargs):
        super().__init__(
            PatchEmbedding(emb_size),
            FlattenHead()
        )

class Proj_eeg(nn.Sequential):
    def __init__(self, embedding_dim=1440, proj_dim=1024, drop_proj=0.5):
        super().__init__(
            nn.Linear(embedding_dim, proj_dim),
            ResidualAdd(nn.Sequential(
                nn.GELU(),
                nn.Linear(proj_dim, proj_dim),
                nn.Dropout(drop_proj),
            )),
            nn.LayerNorm(proj_dim),
        )

class ATMS(nn.Module):    
    def __init__(self, num_channels=63, sequence_length=250, num_subjects=2, num_features=64, num_latents=1024, num_blocks=1):
        super(ATMS, self).__init__()
        default_config = Config()
        self.encoder = iTransformer(default_config)   
        self.subject_wise_linear = nn.ModuleList([nn.Linear(default_config.d_model, sequence_length) for _ in range(num_subjects)])
        self.enc_eeg = Enc_eeg()
        self.proj_eeg = Proj_eeg()        
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        self.loss_func = ClipLoss()       
         
    def forward(self, x, subject_ids):
        x = self.encoder(x, None, subject_ids)
        # print(f'After attention shape: {x.shape}')
        # print("x", x.shape)
        # x = self.subject_wise_linear[0](x)
        # print(f'After subject-specific linear transformation shape: {x.shape}')
        eeg_embedding = self.enc_eeg(x)
        out = self.proj_eeg(eeg_embedding)
        return out  


def extract_id_from_string(s):
    match = re.search(r'\d+$', s)
    if match:
        return int(match.group())
    return None



    

def get_eegfeatures(sub, eegmodel, dataloader, device, text_features_all, img_features_all, k, train):
    eegmodel.eval()
    text_features_all = text_features_all.to(device).float()
    img_features_all = img_features_all.to(device).float()
    total_loss = 0
    correct = 0
    total = 0
    alpha =0.9
    top5_correct = 0
    top5_correct_count = 0

    all_labels = set(range(text_features_all.size(0)))
    top5_acc = 0
    mse_loss_fn = nn.MSELoss()
    ridge_lambda = 0.1
    save_features = True
    features_list = []  # List to store features    
    with torch.no_grad():
        for batch_idx, (eeg_data, labels, text, text_features, img, img_features) in enumerate(dataloader):
            eeg_data = eeg_data.to(device)
            text_features = text_features.to(device).float()
            labels = labels.to(device)
            img_features = img_features.to(device).float()
            
            batch_size = eeg_data.size(0)  # Assume the first element is the data tensor
            subject_id = extract_id_from_string(sub)
            # eeg_data = eeg_data.permute(0, 2, 1)
            subject_ids = torch.full((batch_size,), subject_id, dtype=torch.long).to(device)
            # if not config.insubject:
            #     subject_ids = torch.full((batch_size,), -1, dtype=torch.long).to(device)          
            eeg_features = eeg_model(eeg_data, subject_ids)
            features_list.append(eeg_features.detach().cpu())

        
            logit_scale = eeg_model.logit_scale 
                   
            regress_loss =  mse_loss_fn(eeg_features, img_features)
            # print("eeg_features", eeg_features.shape)
            # print(torch.std(eeg_features, dim=-1))
            # print(torch.std(img_features, dim=-1))
            # l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            # loss = (regress_loss + ridge_lambda * l2_norm)       
            img_loss = eegmodel.loss_func(eeg_features, img_features, logit_scale)
            text_loss = eegmodel.loss_func(eeg_features, text_features, logit_scale)
            contrastive_loss = img_loss
            # loss = img_loss + text_loss

            regress_loss =  mse_loss_fn(eeg_features, img_features)
            # print("text_loss", text_loss)
            # print("img_loss", img_loss)
            # print("regress_loss", regress_loss)            
            # l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            # loss = (regress_loss + ridge_lambda * l2_norm)       
            loss = alpha * regress_loss *10 + (1 - alpha) * contrastive_loss*10
            # print("loss", loss)
            total_loss += loss.item()
            
            for idx, label in enumerate(labels):

                possible_classes = list(all_labels - {label.item()})
                selected_classes = random.sample(possible_classes, k-1) + [label.item()]
                selected_img_features = img_features_all[selected_classes]
                

                logits_img = logit_scale * eeg_features[idx] @ selected_img_features.T
                # logits_text = logit_scale * eeg_features[idx] @ selected_text_features.T
                # logits_single = (logits_text + logits_img) / 2.0
                logits_single = logits_img
                # print("logits_single", logits_single.shape)

                # predicted_label = selected_classes[torch.argmax(logits_single).item()]
                predicted_label = selected_classes[torch.argmax(logits_single).item()] # (n_batch, ) \in {0, 1, ..., n_cls-1}
                if predicted_label == label.item():
                    correct += 1        
                total += 1

        features_tensor = torch.cat(features_list, dim=0)
        if save_features:
            print("features_tensor", features_tensor.shape)
            torch.save(features_tensor.cpu(), f"models/generated_mine_2/ATM_S_eeg_features_{sub}-train.pt" if train else f"models/generated_mine_2/ATM_S_eeg_features_{sub}-test.pt")  # Save features as .pt file
    average_loss = total_loss / (batch_idx+1)
    accuracy = correct / total
    return average_loss, accuracy, labels, features_tensor.cpu()

from IPython.display import Image, display
config = {
    "data_path": os.path.join(data_dir, "Preprocessed_data_250Hz"),
    "project": "atms_reconstruction",
    "entity": "gasparyanartur",
    "name": "lr=3e-4_img_pos_pro_eeg",
    "lr": 3e-4,
    "epochs": 50,
    "batch_size": 1024,
    "logger": True,
    "encoder_type":'ATMS',
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = config['data_path']
features_dir = os.path.join(data_dir, "features")
emb_img_test = torch.load(os.path.join(features_dir, 'ViT-H-14_features_test.pt'))
emb_img_train = torch.load(os.path.join(features_dir, 'ViT-H-14_features_train.pt'))

eeg_model = ATMS()
print('number of parameters:', sum([p.numel() for p in eeg_model.parameters()]))

#####################################################################################

# eeg_model.load_state_dict(torch.load("/home/ldy/Workspace/Reconstruction/models/contrast/sub-08/01-30_00-44/40.pth"))
#cp = torch.load("models/contrast/ATMS/sub-08/09-29_10-36/40.pth")
#
#eeg_model.load_state_dict(cp)


eeg_model.load_state_dict(filtered_state_dict)
eeg_model = eeg_model.to(device)
sub = 'sub-08'
#####################################################################################



Found 1 cuda devices. Using cuda:0 device.


/home/gasparyanartur/miniconda3/envs/BCI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number of parameters: 3202413


In [5]:
#####################################################################################
train_dataset = EEGDataset(data_path, subjects= [sub], train=True)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=0)
text_features_test_all = train_dataset.text_features
img_features_test_all = train_dataset.img_features

train_loss, train_accuracy, labels, eeg_features_train = get_eegfeatures(sub, eeg_model, train_loader, device, text_features_test_all, img_features_test_all,k=200, train=True)
print(f" - Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")


test_dataset = EEGDataset(data_path, subjects= [sub], train=False)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=0)
text_features_test_all = test_dataset.text_features
img_features_test_all = test_dataset.img_features
test_loss, test_accuracy,labels, eeg_features_test = get_eegfeatures(sub, eeg_model, test_loader, device, text_features_test_all, img_features_test_all,k=200, train=False)
print(f" - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
#####################################################################################

self.subjects ['sub-08']
exclude_subject None
data_tensor torch.Size([66160, 63, 250])
Data tensor shape: torch.Size([66160, 63, 250]), label tensor shape: torch.Size([66160]), text length: 1654, image length: 16540
Finished loading features
features_tensor torch.Size([66160, 1024])
 - Train Loss: 13.9527, Train Accuracy: 0.0045
self.subjects ['sub-08']
exclude_subject None
Data tensor shape: torch.Size([200, 63, 250]), label tensor shape: torch.Size([200]), text length: 200, image length: 200
Finished loading features
features_tensor torch.Size([200, 1024])
 - Test Loss: 10.9291, Test Accuracy: 0.5200


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import open_clip
from matplotlib.font_manager import FontProperties

import sys
from diffusion_prior import *
from custom_pipeline import *
# os.environ["CUDA_VISIBLE_DEVICES"] = "5" 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
models_dir = "models/generated_mine_2"

emb_img_train_4 = emb_img_train["img_features"].view(1654,10,1,1024).repeat(1,1,4,1).view(-1,1024)
emb_eeg = torch.load(os.path.join(models_dir, 'ATM_S_eeg_features_sub-08-train.pt'))
emb_eeg_test = torch.load(os.path.join(models_dir, 'ATM_S_eeg_features_sub-08-test.pt'))

In [8]:
TRAIN_DIFFUSION_PRIOR = True
if TRAIN_DIFFUSION_PRIOR:
    assert "original" not in models_dir

model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
save_path = os.path.join(models_dir, f'{config["encoder_type"]}/{sub}/{model_name}.pt')

dataset = EmbeddingDataset(
    c_embeddings=eeg_features_train, h_embeddings=emb_img_train_4, 
    # h_embeds_uncond=h_embeds_imgnet
)
dl = DataLoader(dataset, batch_size=1024, shuffle=True, num_workers=64)
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
# number of parameters
print(sum(p.numel() for p in diffusion_prior.parameters() if p.requires_grad))
pipe = Pipe(diffusion_prior, device=device)

if TRAIN_DIFFUSION_PRIOR:
    pipe.train(dl, num_epochs=150, learning_rate=1e-3) # to 0.142 


    directory = os.path.dirname(save_path)

    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    torch.save(pipe.diffusion_prior.state_dict(), save_path)

else:
    pipe.diffusion_prior.load_state_dict(torch.load(save_path, map_location=device))

pipe.diffusion_prior.eval()
pipe.diffusion_prior.requires_grad_(False)

9675648


/home/gasparyanartur/miniconda3/envs/BCI/lib/python3.10/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch: 0, loss: 1.1385870438355665
epoch: 1, loss: 0.8879161623808054
epoch: 2, loss: 0.6533835484431341
epoch: 3, loss: 0.4864240233714764
epoch: 4, loss: 0.3616069041765653
epoch: 5, loss: 0.2686834990978241
epoch: 6, loss: 0.2122892065690114
epoch: 7, loss: 0.1805946517449159
epoch: 8, loss: 0.15962580809226404
epoch: 9, loss: 0.1423263056920125
epoch: 10, loss: 0.12653853652568964
epoch: 11, loss: 0.11255428561797509
epoch: 12, loss: 0.10002006854002292
epoch: 13, loss: 0.0890115322975012
epoch: 14, loss: 0.07955897049261973
epoch: 15, loss: 0.07100760615788973
epoch: 16, loss: 0.06407074263462653
epoch: 17, loss: 0.058054979202839044
epoch: 18, loss: 0.052686737707028024
epoch: 19, loss: 0.04835003786362135
epoch: 20, loss: 0.044963942353542036
epoch: 21, loss: 0.04144980259812795
epoch: 22, loss: 0.03895826907112048
epoch: 23, loss: 0.03638199871549239
epoch: 24, loss: 0.03444103724681414
epoch: 25, loss: 0.03267107256329976
epoch: 26, loss: 0.031021901965141298
epoch: 27, loss: 

DiffusionPriorUNet(
  (time_proj): Timesteps()
  (input_layer): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
  )
  (encode_time_embedding): ModuleList(
    (0): TimestepEmbedding(
      (linear_1): Linear(in_features=512, out_features=1024, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (1): TimestepEmbedding(
      (linear_1): Linear(in_features=512, out_features=512, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=512, out_features=512, bias=True)
    )
    (2): TimestepEmbedding(
      (linear_1): Linear(in_features=512, out_features=256, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): TimestepEmbedding(
      (linear_1): Linear(in_features=512, out_features=128, bias=True)
      (act): SiLU()
      (linear_2): Linear(in

In [9]:
# Generate test embeddings

test_embeddings = []
with torch.no_grad():
    for k in tqdm(list(range(200)), desc="Generating test embeddings..."):
        eeg_embeds = emb_eeg_test[k:k+1]
        h = pipe.generate(c_embeds=eeg_embeds, num_inference_steps=50, guidance_scale=5.0)
        test_embeddings.append(h.detach().cpu())
    test_embeddings = torch.cat(test_embeddings)
print(test_embeddings.shape)

Generating test embeddings...: 100%|██████████| 200/200 [00:51<00:00,  3.88it/s]

torch.Size([200, 1024])


In [11]:
print("All tensors and modules:")
import itertools as it
to_delete = []
for k, v in locals().items():
    if isinstance(v, (torch.Tensor, torch.nn.Module)):
        v.to("cpu")
        print(k.replace('\n', ''))
    if k.startswith("_"):
        to_delete.append((k, v))
        
for (k, v) in to_delete:
    try:
        del v
    except NameError:
        pass

import gc
try:
    del eeg_model
except NameError:
    pass
try:
    del generator
except NameError:
    pass

try:
    del pipe
except NameError:
    pass

try:
    del diffusion_prior
except NameError:
    pass

gc.collect()
torch.cuda.empty_cache()

All tensors and modules:
_
eeg_model
text_features_test_all
img_features_test_all
labels
eeg_features_train
eeg_features_test
emb_img_train_4
emb_eeg
emb_eeg_test
diffusion_prior
_8
test_embeddings
eeg_embeds
h


In [12]:

from PIL import Image
import os
import diffusers 

# Not sure why they've added this, but it makes the model fail
if "http_proxy" in os.environ:
    os.environ.pop("http_proxy")
if "https_proxy" in os.environ:
    os.environ.pop("https_proxy")
os.environ["HF_HUB_OFFLINE"] = "False"


# Assuming generator.generate returns a PIL Image
generator = Generator4Embeds(num_inference_steps=4, device=device)

diffusers.utils.logging.disable_progress_bar()

directory = f"generated_imgs/mine2/{sub}"
os.makedirs(directory, exist_ok=True)
NUM_GEN_PER_IMG = 3
for k in tqdm(list(range(200)), desc="Generating images..."):
    h = test_embeddings[k:k+1].to(dtype=torch.float16)
    text_dir = os.path.join(directory, texts[k])
    os.makedirs(text_dir, exist_ok=True)
    for j in range(NUM_GEN_PER_IMG):
        image = generator.generate(h)
        image.save(os.path.join(text_dir, f'{j}.png'))


Loading generator...
Loading pipe...


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  7.20it/s]


Pipe loaded
	Loading ip adapter...
	IP adapter loaded
Finished loading generator


Generating images...:   0%|          | 0/200 [00:00<?, ?it/s]/home/gasparyanartur/miniconda3/envs/BCI/lib/python3.10/site-packages/diffusers/models/embeddings.py:2587: FutureWarning: You have passed a tensor as `image_embeds`.This is deprecated and will be removed in a future release. Please make sure to update your script to pass `image_embeds` as a list of tensors to suppress this warning.
  deprecate("image_embeds not a list", "1.0.0", deprecation_message, standard_warn=False)
Generating images...: 100%|██████████| 200/200 [06:16<00:00,  1.88s/it]


In [ ]:
print(h.shape)

torch.Size([1, 1024])
